In [4]:
sample_titles = [
    #
    # (channel_name, video_title)
    #
    
    ('Juice WRLD', 'Juice WRLD & The Weeknd - Smile (Official Video)'),
    ('Drake', 'Laugh Now Cry Later'),
    ('Stone Music Entertainment', "ATEEZ (에이티즈) - 'INCEPTION' MV"),
    ('MrSuicideSheep', 'Stonefox - The Garden')
]

#### Objective

Providing channel name & video title to this class should produce search query(ies) that our `SpotifySearch` uses. Maximise result accuracy.


Accurate result is defined as a perfect match for the song on S